## Goal
The goal of this exploration is to first visualize and understand what the key landmarks represent. Then format the data such that a simple model can be trained

### Data Exploration

In [8]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import plotly.offline as py

data = pd.read_csv('data/train.csv', delimiter=',',encoding='UTF-8')
pd.set_option('display.max_columns', None)
data.head(3)


,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816862427,0,hentaihubs.com


In [9]:
def get_order(landmark_type, landmark_index):
    if landmark_type == 'face':
        return landmark_index + 75
    elif landmark_type == 'pose':
        return landmark_index + 42
    elif landmark_type == 'left_hand':
        return landmark_index + 21
    else:
        return landmark_index


def convert_paquet_to_3D_style(sequence):
    
    #sequence.reset_index(drop=True)
    sequence.reset_index(inplace=False)
    
    num_frames = sequence.shape[0]
    num_landmarks = int((sequence.shape[1] -1 ) /3) # -1 because of frame column
    shape = (num_landmarks,3,num_frames)
    new_style_df = np.zeros(shape)
  
    for index, row in sequence.iterrows():
        i = 0
        for column in list(sequence.columns)[1:]:
            i += 1
            parts = column.split("_")
            landmark_index = int(parts[-1])
            
            axis = parts[0]
            axis_num = 0

            frame_number = int(row['frame'])

            if axis == 'y':
                axis_num = 1 
            elif axis == 'z':
                axis_num = 2

            if len(parts) == 4:
                landmark_type = parts[1] + "_" + parts[2]
                unqiue_landmark_number = get_order(landmark_type,landmark_index)
                
                new_style_df[unqiue_landmark_number, int(axis_num), frame_number] = row[f"{axis}_{landmark_type}_{landmark_index}"]
            else:
                landmark_type = parts[1]
                unqiue_landmark_number = get_order(landmark_type,landmark_index)
                
                new_style_df[unqiue_landmark_number, int(axis_num), frame_number] = row[f"{axis}_{landmark_type}_{landmark_index}"]
    
    return new_style_df

In [61]:
def ShowAnimation_3D(landmarks3D):
    landmarks3D = convert_paquet_to_3D_style(landmarks3D)
    #landmarks3D = map_new_to_old_style(landmarks3D)
    #landmarks3D['plot_order'] = landmarks3D.apply(lambda row: assign_order(row), axis=1)
    #landmarks3D = landmarks3D.copy()
    #landmarks3D[:,0,:] = -landmarks3D[:,0,:]
    landmarks3D[np.isnan(landmarks3D)] = 0 # I think this may mess up the coordinate system, we'll see 

    # xMin = np.nanmin(landmarks3D[:,0,:]) -1
    # xMax = np.nanmax(landmarks3D[:,0,:])+1
    # yMin = np.nanmin(landmarks3D[:,1,:]) -1
    # yMax = np.nanmax(landmarks3D[:,1,:])+1
    # zMin = np.nanmin(landmarks3D[:,2,:])-1
    # zMax = np.nanmax(landmarks3D[:,2,:])+1

    xMin = 0
    xMax = 1
    yMin = 0
    yMax = 1
    zMin = -0.5
    zMax = 0.5
    
    boxCorners = np.array([[xMin,yMin,zMin],
                           [xMin,yMin,zMax],
                           [xMin,yMax,zMin],
                           [xMin,yMax,zMax],
                           [xMax,yMin,zMin],
                           [xMax,yMin,zMax],
                           [xMax,yMax,zMin],
                           [xMax,yMax,zMax]])


    
    traversalOrder = [0,1,3,2,0,4,6,2,6,7,3,7,5,1,5,4]
    boxTraceCoords = np.zeros((len(traversalOrder),3))
    for i, corner in enumerate(traversalOrder):
        boxTraceCoords[i,:] = boxCorners[corner,:]

# np.r_[0:3,7:10]
    right_hand_lines = [np.r_[0:5],
        np.r_[0,5:9],
        np.r_[0,9:13],
        np.r_[0,13:17],
        np.r_[0,17:21]]
    
    trace1 = go.Scatter3d(name='Right Hand 1', 
                          x=landmarks3D[:,0,1][0:5] 
                            ,y=landmarks3D[:,1,1][0:5]
                            ,z=landmarks3D[:,2,1][0:5],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 1))
    
    trace2   = go.Scatter3d(name='Right Hand 2',
                             x = landmarks3D[:,0,1][right_hand_lines[1]]
                            ,y=landmarks3D[:,1,1][right_hand_lines[1]]
                            ,z= landmarks3D[:,2,1][right_hand_lines[1]],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 1))
    
    trace3   = go.Scatter3d(name='Right Hand 3',
                             x = landmarks3D[:,0,1][right_hand_lines[2]]
                            ,y=landmarks3D[:,1,1][right_hand_lines[2]]
                            ,z= landmarks3D[:,2,1][right_hand_lines[2]],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 1))
                            
    trace4   = go.Scatter3d(name='Right Hand 4',
                             x = landmarks3D[:,0,1][right_hand_lines[3]]
                            ,y=landmarks3D[:,1,1][right_hand_lines[3]]
                            ,z= landmarks3D[:,2,1][right_hand_lines[3]],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 1))
    trace5   = go.Scatter3d(name='Right Hand 5',
                             x = landmarks3D[:,0,1][right_hand_lines[4]]
                            ,y=landmarks3D[:,1,1][right_hand_lines[4]]
                            ,z= landmarks3D[:,2,1][right_hand_lines[4]],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 1))
    # trace3   = go.Scatter3d(name='Left Eyebrow',x=landmarks3D[:,0,1][22:27],y=landmarks3D[:,1,1][22:27],z=landmarks3D[:,2,1][22:27],
    #                         mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 5))
    
    # trace4   = go.Scatter3d(name='Nose Ridge',x=landmarks3D[:,0,1][27:31],y=landmarks3D[:,1,1][27:31],z=landmarks3D[:,2,1][27:31],
    #                         mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    # trace5   = go.Scatter3d(name='Nose Base',x=landmarks3D[:,0,1][31:36],y=landmarks3D[:,1,1][31:36],z=landmarks3D[:,2,1][31:36],
    #                         mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    # trace6   = go.Scatter3d(name='Right Eye', x=landmarks3D[:,0,1][36:42],y=landmarks3D[:,1,1][36:42],z=landmarks3D[:,2,1][36:42],
    #                         mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    # trace7   = go.Scatter3d(name='Left Eye', x=landmarks3D[:,0,1][42:48],y=landmarks3D[:,1,1][42:48],z=landmarks3D[:,2,1][42:48],
    #                         mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    # trace8   = go.Scatter3d(name='Outer Mouth', x=landmarks3D[:,0,1][48:60],y=landmarks3D[:,1,1][48:60],z=landmarks3D[:,2,1][48:60],
    #                         mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    # trace9   = go.Scatter3d(name='Inner Mouth', x=landmarks3D[:,0,1][60:68],y=landmarks3D[:,1,1][60:68],z=landmarks3D[:,2,1][60:68],
    #                         mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
        
    boxTrace = go.Scatter3d(name='boundingBox', x=boxTraceCoords[:,0],y=boxTraceCoords[:,1],z=boxTraceCoords[:,2],
                            mode='lines+markers',marker=dict(color = 'red', opacity=1.0,size = 1))
    
    data = [trace1, trace2,trace3, trace4, trace5,boxTrace]# , trace6, trace7, trace8, trace9, boxTrace]
    # data = [boxTrace]# trace3, trace4, trace5, trace6, trace7, trace8, trace9, boxTrace]

    
    mfr = []
    for t in range(len(landmarks3D[1,1,:])):

        mfr.append({'data' :[{'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][0:5]
                              ,'y':landmarks3D[:,1,t][0:5]
                              ,'z':landmarks3D[:,2,t][0:5]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][right_hand_lines[1]]
                              ,'y':landmarks3D[:,1,t][right_hand_lines[1]]
                              ,'z':landmarks3D[:,2,t][right_hand_lines[1]]
                              },
                              {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][right_hand_lines[2]]
                              ,'y':landmarks3D[:,1,t][right_hand_lines[2]]
                              ,'z':landmarks3D[:,2,t][right_hand_lines[2]]
                              },
                              {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][right_hand_lines[3]]
                              ,'y':landmarks3D[:,1,t][right_hand_lines[3]]
                              ,'z':landmarks3D[:,2,t][right_hand_lines[3]]
                              },
                              {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][right_hand_lines[4]]
                              ,'y':landmarks3D[:,1,t][right_hand_lines[4]]
                              ,'z':landmarks3D[:,2,t][right_hand_lines[4]]
                              },
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][22:27],'y':landmarks3D[:,1,t][22:27],'z':landmarks3D[:,2,t][22:27]},
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][27:31],'y':landmarks3D[:,1,t][27:31],'z':landmarks3D[:,2,t][27:31]},
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][31:36],'y':landmarks3D[:,1,t][31:36],'z':landmarks3D[:,2,t][31:36]},
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][36:42],'y':landmarks3D[:,1,t][36:42],'z':landmarks3D[:,2,t][36:42]},
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][42:48],'y':landmarks3D[:,1,t][42:48],'z':landmarks3D[:,2,t][42:48]},
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][48:60],'y':landmarks3D[:,1,t][48:60],'z':landmarks3D[:,2,t][48:60]},
                            #  {'type' : "scatter3d",'mode':'lines+markers',
                            #   'x':landmarks3D[:,0,t][60:68],'y':landmarks3D[:,1,t][60:68],'z':landmarks3D[:,2,t][60:68]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                             'x':boxTraceCoords[:,0],'y':boxTraceCoords[:,1],'z':boxTraceCoords[:,2]}]})
                                 
    
    layout = go.Layout(width=800, height=800, title='3D Face Shape Animation',
                       scene=dict(camera=dict(up     = dict(x= 0, y=-1.0, z=0),
                                              center = dict(x= 0.5, y= 0.5, z=0),
                                              eye    = dict(x= 0, y= 0.7, z=2.5),
                                             )
                                 ),
                        updatemenus=[dict(type='buttons', showactive=False,
                                            y=1,
                                            x=1,
                                            xanchor='right',
                                            yanchor='top',
                                            pad=dict(t=0, r=10),
                                            buttons=[dict(
                                                        label='Play Animation',
                                                        method='animate',
                                                        args=[None, dict(frame       = dict(duration=0.04, redraw=True), 
                                                                         transition  = dict(duration=0),
                                                                         fromcurrent = True,
                                                                         mode = 'immediate'
                                                                        )
                                                             ]
                                                         )
                                                    ]
                                           )
                                      ]
                      )
                                                    
    fig = dict(data=data, layout=layout, frames=mfr)
    # fig = dict(data=data, frames=mfr)
    py.iplot(fig)

In [62]:
ShowAnimation_3D(phrase_keypoints)

In [7]:
# Most of this function is borrowed from 
# https://www.kaggle.com/code/sharifi76/eda-motion-visualization-fingerspelling


def map_new_to_old_style(sequence):
    types = []
    landmark_indexes = []

    for column in list(sequence.columns)[1:544]:
        parts = column.split("_")
        if len(parts) == 4:
            landmark_type = parts[1] + "_" + parts[2]
            types.append(landmark_type)
        else:
            landmark_type = parts[1]
            types.append(landmark_type)

        landmark_indexes.append(int(parts[-1]))

    data = {
        "frame": [],
        "type": [],
        "landmark_index": [],
        "x": [],
        "y": [],
        "z": []
    }

    for index, row in sequence.iterrows():
        data["frame"] += [int(row.frame)]*543
        data["type"] += types
        data["landmark_index"] += landmark_indexes

        for _type, landmark_index in zip(types, landmark_indexes):
            data["x"].append(row[f"x_{_type}_{landmark_index}"])
            data["y"].append(row[f"y_{_type}_{landmark_index}"])
            data["z"].append(row[f"z_{_type}_{landmark_index}"])

    return pd.DataFrame.from_dict(data)

# assign desired colors to landmarks
def assign_color(row):
    if row == 'face':
        return 'red'
    elif 'hand' in row:
        return 'dodgerblue'
    else:
        return 'green'

# specifies the plotting order
def assign_order(row):
    if row.type == 'face':
        return row.landmark_index + 101
    elif row.type == 'pose':
        return row.landmark_index + 30
    elif row.type == 'left_hand':
        return row.landmark_index + 80
    else:
        return row.landmark_index

def visualise2d_landmarks(parquet_df, title="",inter_frame_delay : int = 100):
    connections = [  
        [0, 1, 2, 3, 4,],
        [0, 5, 6, 7, 8],
        [0, 9, 10, 11, 12],
        [0, 13, 14, 15, 16],
        [0, 17, 18, 19, 20],

        
        [38, 36, 35, 34, 30, 31, 32, 33, 37],
        [40, 39],
        [52, 46, 50, 48, 46, 44, 42, 41, 43, 45, 47, 49, 45, 51],
        [42, 54, 56, 58, 60, 62, 58],
        [41, 53, 55, 57, 59, 61, 57],
        [54, 53],

        
        [80, 81, 82, 83, 84, ],
        [80, 85, 86, 87, 88],
        [80, 89, 90, 91, 92],
        [80, 93, 94, 95, 96],
        [80, 97, 98, 99, 100], ]

    parquet_df = map_new_to_old_style(parquet_df)
    frames = sorted(set(parquet_df.frame))
    first_frame = min(frames)
    parquet_df['color'] = parquet_df.type.apply(lambda row: assign_color(row))
    parquet_df['plot_order'] = parquet_df.apply(lambda row: assign_order(row), axis=1)
    first_frame_df = parquet_df[parquet_df.frame == first_frame].copy()
    first_frame_df = first_frame_df.sort_values(["plot_order"]).set_index('plot_order')


    frames_l = []
    for frame in frames:
        filtered_df = parquet_df[parquet_df.frame == frame].copy()
        filtered_df = filtered_df.sort_values(["plot_order"]).set_index("plot_order")
        traces = [go.Scatter(
            x=filtered_df['x'],
            y=filtered_df['y'],
            mode='markers',
            marker=dict(
                color=filtered_df.color,
                size=9))]

        for i, seg in enumerate(connections):
            trace = go.Scatter(
                    x=filtered_df.loc[seg]['x'],
                    y=filtered_df.loc[seg]['y'],
                    mode='lines',
            )
            traces.append(trace)
        frame_data = go.Frame(data=traces, traces = [i for i in range(17)])
        frames_l.append(frame_data)

    traces = [go.Scatter(
        x=first_frame_df['x'],
        y=first_frame_df['y'],
        mode='markers',
        marker=dict(
            color=first_frame_df.color,
            size=9
        )
    )]
    for i, seg in enumerate(connections):
        trace = go.Scatter(
            x=first_frame_df.loc[seg]['x'],
            y=first_frame_df.loc[seg]['y'],
            mode='lines',
            line=dict(
                color='black',
                width=2
            )
        )
        traces.append(trace)
    
    fig = go.Figure(
            data=traces,
            frames=frames_l
        )

    fig.update_layout(
        width=500,
        height=800,
        scene={
            'aspectmode': 'data',
        },
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": inter_frame_delay,
                                                  "redraw": True},
                                        "fromcurrent": True,
                                        "transition": {"duration": 0}}],
                        "label": "&#9654;",
                        "method": "animate",
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                          "mode": "immediate",
                                          "transition": {"duration": 0}}],
                        "label": "&#9612;&#9612;",
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 100, "t": 100},
                "font": {"size":20},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
    )
    camera = dict(
        up=dict(x=0, y=-1, z=0),
        eye=dict(x=0, y=0, z=2.5)
    )
    fig.update_layout(title_text=title, title_x=0.5)
    fig.update_layout(scene_camera=camera, showlegend=False)
    fig.update_layout(xaxis = dict(visible=False),
            yaxis = dict(visible=False),
    )
    fig.update_yaxes(autorange="reversed")

    fig.show()

Okay, because of the size of the data set (135GB), we have only downloaded a subset of videos to play around with. Let's get the names of those files 

In [11]:
import os 
landmark_folder = 'train_landmarks/'
selected_files = os.listdir(path='./data/train_landmarks')
selected_files = [landmark_folder + f  for f in selected_files ]
print(selected_files)

['train_landmarks/105143404.parquet', 'train_landmarks/1021040628.parquet', 'train_landmarks/1019715464.parquet']


Okay great! Now let's filter for only the training data of those files

In [12]:
rslt_df = data[data['path'].isin(selected_files)]
rslt_df.head()

,path,file_id,sequence_id,participant_id,phrase
1000,train_landmarks/105143404.parquet,105143404,1784552841,188,https://www.xn--pearol-xwa.org
1001,train_landmarks/105143404.parquet,105143404,1784574169,89,+44-645-14-4280-31
1002,train_landmarks/105143404.parquet,105143404,1784587545,219,dan navarro
1003,train_landmarks/105143404.parquet,105143404,1784630915,81,mephimle.com/shine-papeleria/
1004,train_landmarks/105143404.parquet,105143404,1784672555,21,facartes.uniandes.edu.co


Okay great! Now we only have the landmarks for the videos we have downloaded. Let's visualize the first phrase we have. This one has a sequence id of ```1784552841```

In [13]:
phrase_string =rslt_df.loc[rslt_df["sequence_id"] == 1784552841].phrase.values[0]
phrases = pd.read_parquet("./data/train_landmarks/105143404.parquet")
phrases.head()



frame  x_face_0  x_face_1  x_face_2  x_face_3  x_face_4  \
sequence_id                                                            
1784552841       0  0.492320  0.482797  0.485312  0.471907  0.481505   
1784552841       1  0.492477  0.481592  0.483950  0.470245  0.480239   
1784552841       2  0.487783  0.477862  0.480728  0.466654  0.476475   
1784552841       3  0.486822  0.476097  0.479721  0.463738  0.474360   
1784552841       4  0.489659  0.476537  0.479497  0.465263  0.475073   

             x_face_5  x_face_6  x_face_7  x_face_8  x_face_9  x_face_10  \
sequence_id                                                                
1784552841   0.480572  0.478806  0.412168  0.475747  0.473914   0.468449   
1784552841   0.479128  0.476890  0.411729  0.473802  0.471986   0.466699   
1784552841   0.475539  0.474092  0.411478  0.471471  0.469825   0.465527   
1784552841   0.473089  0.470957  0.402131  0.467684  0.465635   0.460006   
1784552841   0.474147  0.472896  0.403360  0.470319  0.468619   0.464267   

             x_face_11  x_face_12  x_face_13  x_face_14  x_face_15  x_face_16  \
sequence_id                                                                     
1784552841    0.492557   0.492694   0.492736   0.495522   0.495859   0.496199   
1784552841    0.492669   0.492506   0.492104   0.494531   0.495105   0.495674   
1784552841    0.488124   0.488256   0.488210   0.491264   0.491905   0.492595   
1784552841    0.487157   0.487449   0.487573   0.491016   0.491549   0.492241   
1784552841    0.490167   0.490466   0.490525   0.492352   0.493133   0.493934   

             x_face_17  x_face_18  x_face_19  x_face_20  x_face_21  x_face_22  \
sequence_id                                                                     
1784552841    0.496333   0.495450   0.483891   0.476885   0.366187   0.438706   
1784552841    0.495871   0.494721   0.482697   0.475540   0.364737   0.437623   
1784552841    0.493030   0.492126   0.479165   0.472212   0.363854   0.437069   
1784552841    0.492776   0.493472   0.477711   0.470451   0.360055   0.429539   
1784552841    0.494442   0.491559   0.477901   0.470505   0.364401   0.431109   

             x_face_23  x_face_24  x_face_25  x_face_26  x_face_27  x_face_28  \
sequence_id                                                                     
1784552841    0.429107   0.419820   0.407504   0.446882   0.421906   0.432598   
1784552841    0.428329   0.419219   0.406914   0.445464   0.421566   0.431806   
1784552841    0.428524   0.419697   0.406399   0.443944   0.421914   0.431257   
1784552841    0.419979   0.410580   0.397463   0.437642   0.413852   0.424427   
1784552841    0.421931   0.412644   0.398930   0.438621   0.413997   0.424421   

             x_face_29  x_face_30  x_face_31  x_face_32  x_face_33  x_face_34  \
sequence_id                                                                     
1784552841    0.411900   0.405584   0.398646   0.459111   0.408703   0.366985   
1784552841    0.411842   0.405561   0.397729   0.457964   0.408139   0.364170   
1784552841    0.412577   0.406291   0.397646   0.454691   0.407607   0.361841   
1784552841    0.403712   0.397068   0.389296   0.455414   0.398421   0.360782   
1784552841    0.404102   0.397782   0.391520   0.453744   0.399668   0.363039   

             x_face_35  x_face_36  x_face_37  x_face_38  x_face_39  x_face_40  \
sequence_id                                                                     
1784552841    0.386211   0.434219   0.482279   0.484460   0.473351   0.468210   
1784552841    0.384629   0.432188   0.481004   0.482661   0.469964   0.462771   
1784552841    0.383577   0.428182   0.476424   0.478621   0.466044   0.459582   
1784552841    0.377650   0.425578   0.475387   0.477632   0.465126   0.458626   
1784552841    0.378913   0.426966   0.478248   0.480779   0.467876   0.461407   

             x_face_41  x_face_42  x_face_43  x_face_44  x_face_45  x_face_46  \
sequence_id                                           

Okay now we have everything in that parquet. Let's just look at our one phrase

In [14]:
phrase_keypoints = phrases.loc[1784552841]
# phrase_keypoints = phrase_keypoints.reset_index(inplace=False)
phrase_keypoints.head()

frame  x_face_0  x_face_1  x_face_2  x_face_3  x_face_4  \
sequence_id                                                            
1784552841       0  0.492320  0.482797  0.485312  0.471907  0.481505   
1784552841       1  0.492477  0.481592  0.483950  0.470245  0.480239   
1784552841       2  0.487783  0.477862  0.480728  0.466654  0.476475   
1784552841       3  0.486822  0.476097  0.479721  0.463738  0.474360   
1784552841       4  0.489659  0.476537  0.479497  0.465263  0.475073   

             x_face_5  x_face_6  x_face_7  x_face_8  x_face_9  x_face_10  \
sequence_id                                                                
1784552841   0.480572  0.478806  0.412168  0.475747  0.473914   0.468449   
1784552841   0.479128  0.476890  0.411729  0.473802  0.471986   0.466699   
1784552841   0.475539  0.474092  0.411478  0.471471  0.469825   0.465527   
1784552841   0.473089  0.470957  0.402131  0.467684  0.465635   0.460006   
1784552841   0.474147  0.472896  0.403360  0.470319  0.468619   0.464267   

             x_face_11  x_face_12  x_face_13  x_face_14  x_face_15  x_face_16  \
sequence_id                                                                     
1784552841    0.492557   0.492694   0.492736   0.495522   0.495859   0.496199   
1784552841    0.492669   0.492506   0.492104   0.494531   0.495105   0.495674   
1784552841    0.488124   0.488256   0.488210   0.491264   0.491905   0.492595   
1784552841    0.487157   0.487449   0.487573   0.491016   0.491549   0.492241   
1784552841    0.490167   0.490466   0.490525   0.492352   0.493133   0.493934   

             x_face_17  x_face_18  x_face_19  x_face_20  x_face_21  x_face_22  \
sequence_id                                                                     
1784552841    0.496333   0.495450   0.483891   0.476885   0.366187   0.438706   
1784552841    0.495871   0.494721   0.482697   0.475540   0.364737   0.437623   
1784552841    0.493030   0.492126   0.479165   0.472212   0.363854   0.437069   
1784552841    0.492776   0.493472   0.477711   0.470451   0.360055   0.429539   
1784552841    0.494442   0.491559   0.477901   0.470505   0.364401   0.431109   

             x_face_23  x_face_24  x_face_25  x_face_26  x_face_27  x_face_28  \
sequence_id                                                                     
1784552841    0.429107   0.419820   0.407504   0.446882   0.421906   0.432598   
1784552841    0.428329   0.419219   0.406914   0.445464   0.421566   0.431806   
1784552841    0.428524   0.419697   0.406399   0.443944   0.421914   0.431257   
1784552841    0.419979   0.410580   0.397463   0.437642   0.413852   0.424427   
1784552841    0.421931   0.412644   0.398930   0.438621   0.413997   0.424421   

             x_face_29  x_face_30  x_face_31  x_face_32  x_face_33  x_face_34  \
sequence_id                                                                     
1784552841    0.411900   0.405584   0.398646   0.459111   0.408703   0.366985   
1784552841    0.411842   0.405561   0.397729   0.457964   0.408139   0.364170   
1784552841    0.412577   0.406291   0.397646   0.454691   0.407607   0.361841   
1784552841    0.403712   0.397068   0.389296   0.455414   0.398421   0.360782   
1784552841    0.404102   0.397782   0.391520   0.453744   0.399668   0.363039   

             x_face_35  x_face_36  x_face_37  x_face_38  x_face_39  x_face_40  \
sequence_id                                                                     
1784552841    0.386211   0.434219   0.482279   0.484460   0.473351   0.468210   
1784552841    0.384629   0.432188   0.481004   0.482661   0.469964   0.462771   
1784552841    0.383577   0.428182   0.476424   0.478621   0.466044   0.459582   
1784552841    0.377650   0.425578   0.475387   0.477632   0.465126   0.458626   
1784552841    0.378913   0.426966   0.478248   0.480779   0.467876   0.461407   

             x_face_41  x_face_42  x_face_43  x_face_44  x_face_45  x_face_46  \
sequence_id                                           

In [8]:
phrase_keypoints.shape

(294, 1630)

Now we want to visualize this data, I've borrowed the visualization function from [EDA+Motion_Visualization-Fingerspelling](https://www.kaggle.com/code/levistringer/eda-motion-visualization-fingerspelling/edit)

In [11]:
visualise2d_landmarks(phrase_keypoints, f"Phrase: {phrase_string}", inter_frame_delay=200)

KeyboardInterrupt: 

To do
* Research different clustering technqiues. I.e 3d different mappings to new letters
* Temporal analysis for splitting up timings between words